# Задание по лекциям 2.

- Не нужно использовать эффективные вычислительные алгоритмы. Нужно использовать те алгоритмы, что были представлены на лекции или практических занятиях. 
- Остальное как обычно: за "похожие" решения всем задействованным 0 баллов; если используете решение из открытого источника — обязательно вставьте ссылку; не удаляйте формулировки; не выкладывайте в открытый источник.
- Можно использовать `numpy.array` для матриц и матричной арифметики и `numpy.linalg` для подсчёта ранга и определителя, для вычисления обратной матрицы, решения СЛУ и т.п. То есть то, что вы уже реализовывали в прошлом семестре, ещё раз реализовывать необязательно. Более того, можно использовать в любом из *заданий по лекциям* функции, реализованные ранее в других *заданиях по лекциям*. Если возникнут сомнения, можно ли использовать ту или иную функцию — лучше сразу поинтересуйтесь у меня.

In [2]:
import numpy as np
import scipy.optimize
import scipy.linalg
from sympy import Matrix

In [3]:
# https://stackoverflow.com/questions/15638650/is-there-a-standard-solution-for-gauss-elimination-in-python
def gauss(A):
    """
    Метод Гаусса
    """
    pl, u = scipy.linalg.lu(A, permute_l=True)
    return scipy.optimize._remove_redundancy._remove_redundancy(u, np.zeros_like(u[:, 0]))[0]

In [4]:
def make_good(v):
    """
    Делит все значения вектора на первое ненулевое значение
    """
    for i, e in enumerate(v):
        if abs(e) > 1e-10:
            return v / e

def make_world_better(A):
    """
    Проходится по массиву векторов и делит каждый из них на первое ненулевое значение, чтобы сделать значения красивее
    """
    new = []
    for i in range(A.shape[0]):
        new.append(make_good(A[i]))
    return np.array(new)

def fsr(A):
    """
    Поиск фундаментальной системы решений СЛУ
    """
    bad_fsr = scipy.linalg.null_space(A).T
    return bad_fsr

$\mathbb{R}^n$ — вещественнозначное пространство вектор-**столбцов** со стандартным скалярным произведением.

**(1 балл) Задание 5.** Реализовать функцию, принимающую на вход набор координат $u = (u_1,\ldots,u_k),\ k\le n$ вектор-столбцов, и выдающую базис $$v = (v_1,\ldots,v_m),\ m \ge n-k,$$
ортогонального дополнения  для линейной оболочки данных векторов.

In [57]:
def concat_orth_basis(U):
    """
    Дополняет до базиса всего пространства (используется в последнем задании)
    """
    # применяем гаусса к векторам U, чтобы выбрать из них линейно независимые
    U = gauss(U)
    # решим систему уравнений (u_1, x) = 0, ..., (u_k, x) = 0 (найдем фср)
    V = fsr(U)
    # вектора x_1, ..., x_p - фср - векторы базиса искомого орт.дополнения
    return np.concatenate([U,V])

def orth_basis(U):
    """
    Решение - выдает базис ортогонального дополнения
    """
    # применяем гаусса к векторам U, чтобы выбрать из них линейно независимые
    U = gauss(U)
    # решим систему уравнений (u_1, x) = 0, ..., (u_k, x) = 0 (найдем фср)
    V = fsr(U)
    # вектора x_1, ..., x_p - фср - векторы базиса искомого орт.дополнения
    return V

In [64]:
U = np.array([
    [1, 1]
])

ob = orth_basis(U)
make_world_better(ob)

array([[ 1., -1.]])

In [65]:
ob[0].dot(U[0])

1.1102230246251565e-16

**(1 балл) Задание 6.** Реализовать функцию, принимающую на вход набор координат $u = (u_1,\ldots,u_k),\ k\le n$ вектор-столбцов, и выдающую ортонормированный базис $$v = (v_1,\ldots,v_m),\ m \ge n-k,$$
для линейной оболочки данных векторов.

In [60]:
# https://gist.github.com/iizukak/1287876

def gs_cofficient(v1, v2):
    return np.dot(v2, v1) / np.dot(v1, v1)

def multiply(cofficient, v):
    return map((lambda x : x * cofficient), v)

def proj(v1, v2):
    return multiply(gs_cofficient(v1, v2) , v1)

# Грамм-шмидт
def gs(X):
    X = gauss(X) # проходимся Гауссом, чтобы исключить линейно зависимые вектора
    Y = []
    for i in range(len(X)):
        temp_vec = X[i]
        for inY in Y :
            proj_vec = proj(inY, X[i])
            temp_vec = np.array(list(map(lambda x, y : x - y, temp_vec, proj_vec)))
        Y.append(temp_vec)
    for i, e in enumerate(Y):
        Y[i] = Y[i] * 1.0 / np.linalg.norm(Y[i])
    return np.array(Y)

In [61]:
gs(np.array([
    [1,1],
    [1,0],
    [1,1]
]))

array([[ 0.70710678,  0.70710678],
       [ 0.70710678, -0.70710678]])

**(1 балл) Задание 7.** Реализовать функцию, принимающую на вход набор координат $u = (u_1,\ldots,u_k),\ k\le n$ линейно независимых вектор-столбцов, и выдающую ортонормированный базис $$v = (v_1,\ldots,v_n)$$
всего пространства, такой что линейная оболочка векторов $(v_1,\ldots,v_k)$ совпадает с линейной оболочкой векторов $(u_1,\ldots,u_k)$.

In [62]:
def make_othonorm_basis(U):
    """
    U - массив векторов (строки - вектора)
    """
    # Ортогонализуем и нормируем полученный набор векторов
    U = gs(U)
    # Дополняем его до базиса всего пространства
    U = make_world_better(concat_orth_basis(U))
    # Опять ортонормируем полученную систему
    U = gs(U)
    return U